In [5]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

from __future__ import division
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
# Any results you write to the current directory are saved as output.

# Based on Blending code from Emanuele Olivetti

"""
Blending {RandomForests, ExtraTrees, GradientBoosting} + stretching to
[0,1]. The blending scheme is related to the idea Jose H. Solorzano
presented here:
http://www.kaggle.com/c/bioresponse/forums/t/1889/question-about-the-process-of-ensemble-learning/10950#post10950
'''You can try this: In one of the 5 folds, train the models, then use
the results of the models as 'variables' in logistic regression over
the validation data of that fold'''. Or at least this is the
implementation of my understanding of that idea :-)

The predictions are saved in test.csv.

Copyright 2012, Emanuele Olivetti.
BSD license, 3 clauses.
"""

import matplotlib.pyplot as plt
from sklearn import cross_validation
from sklearn.ensemble import RandomForestClassifier,ExtraTreesClassifier,AdaBoostClassifier,GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier,ExtraTreeClassifier
from sklearn.linear_model import LogisticRegression
import xgboost as xgb

if __name__ == '__main__':

	df_train = pd.read_csv('train_art.csv')
	df_test = pd.read_csv('test_art.csv')
	# remove columns with 0 variance
	remove = []
	cols = df_train.columns
	for i in range(len(cols)-1):
		if df_train[cols[i]].std() == 0:
			remove.append(cols[i])

	# remove duplicated columns
	for i in range(len(cols)-1):
		v = df_train[cols[i]].values
		for j in range(i+1,len(cols)):
			if np.array_equal(v,df_train[cols[j]].values):
				remove.append(cols[j])	

	df_train.drop(remove, axis=1, inplace=True)
	df_test.drop(remove, axis=1, inplace=True)

	tokeep = ['num_var39_0',  # 0.00031104199066874026
              'ind_var13',  # 0.00031104199066874026
              'num_op_var41_comer_ult3',  # 0.00031104199066874026
              'num_var43_recib_ult1',  # 0.00031104199066874026
              'imp_op_var41_comer_ult3',  # 0.00031104199066874026
              'num_var8',  # 0.00031104199066874026
              'num_var42',  # 0.00031104199066874026
              'num_var30',  # 0.00031104199066874026
              'saldo_var8',  # 0.00031104199066874026
              'num_op_var39_efect_ult3',  # 0.00031104199066874026
              'num_op_var39_comer_ult3',  # 0.00031104199066874026
              'num_var41_0',  # 0.0006220839813374805
              'num_op_var39_ult3',  # 0.0006220839813374805
              'saldo_var13',  # 0.0009331259720062209
              'num_var30_0',  # 0.0009331259720062209
              'ind_var37_cte',  # 0.0009331259720062209
              'ind_var39_0',  # 0.001244167962674961
              'num_var5',  # 0.0015552099533437014
              'ind_var10_ult1',  # 0.0015552099533437014
              'num_op_var39_hace2',  # 0.0018662519440124418
              'num_var22_hace2',  # 0.0018662519440124418
              'num_var35',  # 0.0018662519440124418
              'ind_var30',  # 0.0018662519440124418
              'num_med_var22_ult3',  # 0.002177293934681182
              'imp_op_var41_efect_ult1',  # 0.002488335925349922
              'var36',  # 0.0027993779160186624
              'num_med_var45_ult3',  # 0.003110419906687403
              'imp_op_var39_ult1',  # 0.0037325038880248835
              'imp_op_var39_comer_ult3',  # 0.0037325038880248835
              'imp_trans_var37_ult1',  # 0.004043545878693624
              'num_var5_0',  # 0.004043545878693624
              'num_var45_ult1',  # 0.004665629860031105
              'ind_var41_0',  # 0.0052877138413685845
              'imp_op_var41_ult1',  # 0.0052877138413685845
              'num_var8_0',  # 0.005598755832037325
              'imp_op_var41_efect_ult3',  # 0.007153965785381027
              'num_op_var41_ult3',  # 0.007153965785381027
              'num_var22_hace3',  # 0.008087091757387248
              'num_var4',  # 0.008087091757387248
              'imp_op_var39_comer_ult1',  # 0.008398133748055987
              'num_var45_ult3',  # 0.008709175738724729
              'ind_var5',  # 0.009953343701399688
              'imp_op_var39_efect_ult3',  # 0.009953343701399688
              'num_meses_var5_ult3',  # 0.009953343701399688
              'saldo_var42',  # 0.01181959564541213
              'imp_op_var39_efect_ult1',  # 0.013374805598755831
              'num_var45_hace2',  # 0.014618973561430793
              'num_var22_ult1',  # 0.017107309486780714
              'saldo_medio_var5_ult1',  # 0.017418351477449457
              'saldo_var5',  # 0.0208398133748056
              'ind_var8_0',  # 0.021150855365474338
              'ind_var5_0',  # 0.02177293934681182
              'num_meses_var39_vig_ult3',  # 0.024572317262830483
              'saldo_medio_var5_ult3',  # 0.024883359253499222
              'num_var45_hace3',  # 0.026749611197511663
              'num_var22_ult3',  # 0.03452566096423017
              'saldo_medio_var5_hace3',  # 0.04074650077760498
              'saldo_medio_var5_hace2',  # 0.04292379471228616
              'n0',  # 0.04696734059097978
              'saldo_var30',  # 0.09611197511664074
              'var38',  # 0.1390357698289269
              'var15', 'art_var3', 'art_var36', 'art_var38mc', 'art_logvar38','pca0', 'pca1', 'pca2', 'pca3', 'pca4', 'pca5', 'pca6', 'pca7', 'pca8', 'pca9', 'pca_all0', 'pca_all1', 'pca_all2', 'pca_all3', 'pca_all4', 'pca_all5', 'pca_all6', 'pca_all7', 'pca_all8', 'pca_all9'] 

	#df_train.replace(0, np.nan, True).to_sparse()
	#df_test.replace(0, np.nan, True).to_sparse()
	#0 if np.isnan(i) else 1
	y = df_train.ix[:,-1]; X = pd.DataFrame.as_matrix(df_train[tokeep])
	y_submission = df_test.ix[:,0]; X_submission = pd.DataFrame.as_matrix(df_test[tokeep])
    

In [ ]:
    # Replace -999999 in var3 column with most common value 2 
    # See https://www.kaggle.com/cast42/santander-customer-satisfaction/debugging-var3-999999
    # for details

    shuffle = False
    n_folds = 3
    if shuffle:
        idx = np.random.permutation(y.size)
        X = X[idx]
        y = y[idx]
    ratio = float(np.sum(y == 1)) / np.sum(y==0)
    clfs = [RandomForestClassifier(n_estimators=400, max_depth=5, class_weight='balanced', criterion = 'gini'),
            RandomForestClassifier(n_estimators=400, max_depth=5, class_weight='balanced', criterion = 'entropy'),
            xgb.XGBClassifier(missing=np.nan,
                              nthread = 4,
                              n_estimators=350,
                              max_depth=5,
                              objective= 'binary:logistic',
                              learning_rate = 0.03,
                              colsample_bytree=0.7,
                              subsample = 0.8,
                              min_child_weight = 1,
                              seed = 9)
]

    print ("Creating train and test sets for blending.")
    
    dataset_blend_train = np.zeros((X.shape[0], len(clfs)))
    dataset_blend_test = np.zeros((X_submission.shape[0], len(clfs)))
    skf = cross_validation.StratifiedKFold(y, n_folds, shuffle=True)
    
    for j, clf in enumerate(clfs):
        print (j, clf)
        dataset_blend_test_j = np.zeros((X_submission.shape[0], n_folds))
        for i, (train, testidx) in enumerate(skf):
            print ("Fold", i)
            X_train, y_train = X[train], y[train]
            X_test, y_test = X[testidx], y[testidx]
#            clf.fit(X_train, y_train)
            if j < len(clfs)-1:
                clf.fit(X_train, y_train)
            else:
                clf.fit(X_train, y_train, early_stopping_rounds=50, eval_metric="auc",
                    eval_set=[(X_test, y_test)])
            y_submission = clf.predict_proba(X_test)[:,1]
            dataset_blend_train[testidx, j] = y_submission
            dataset_blend_test_j[:, i] = clf.predict_proba(X_submission)[:,1]
        dataset_blend_test[:,j] = dataset_blend_test_j.mean(axis=1)

    print ("Blending.")
    clf = LogisticRegression()
    clf.fit(dataset_blend_train, y)
    y_submission = clf.predict_proba(dataset_blend_test)[:,1]

    print ("Saving Results.")
    submission = pd.DataFrame({"ID":test.index, "TARGET":y_submission})
    submission.to_csv("submission_RFG_RFE_XGB.csv", index=False)
    
    
    

In [10]:
    print y_submission
    y_test = df_test.ix[:,0]
    ans = pd.DataFrame({'ID': y_test, 'TARGET' : y_submission})
    ans.to_csv('rfe_rfg_xgb'+'.csv', index=False, columns=['ID', 'TARGET'])

[ 0.03921931  0.03684926  0.01121025 ...,  0.00672753  0.04335623
  0.00688674]
